<a href="https://colab.research.google.com/github/mlignoat/mod2_SparkColab/blob/main/Spark_Install.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [16]:
!pip install pyspark==3.3.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [18]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz

In [19]:
!tar xf spark-3.3.2-bin-hadoop3.tgz

In [20]:
!pip install -q findspark

In [25]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

In [26]:
import findspark
findspark.init()

In [27]:
from pyspark.sql import SparkSession

In [28]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('Iniciando com Spark') \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [29]:
spark

In [90]:
df_titles = spark.read.csv('title_basics.tsv', header=True,inferSchema=True, sep='\t')

In [102]:
df_ratings = spark.read.csv('title_ratings.tsv', header=True,inferSchema=True, sep='\t')

In [189]:
df_title_ratings = df_titles.join(df_ratings, on='tconst')

In [104]:
df_titles.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



In [92]:
df_titles.createOrReplaceTempView('tb_titles')

In [103]:
df_ratings.printSchema()

root
 |-- tconst: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)



In [55]:
df_ratings.filter("numVotes > 0").show()

+---------+-------------+--------+
|   tconst|averageRating|numVotes|
+---------+-------------+--------+
|tt0000001|          5.7|    1809|
|tt0000002|          6.0|     233|
|tt0000003|          6.5|    1560|
|tt0000004|          6.1|     152|
|tt0000005|          6.2|    2383|
|tt0000006|          5.1|     157|
|tt0000007|          5.4|     746|
|tt0000008|          5.5|    1965|
|tt0000009|          5.8|     189|
|tt0000010|          6.9|    6530|
|tt0000011|          5.2|     323|
|tt0000012|          7.4|   11188|
|tt0000013|          5.8|    1726|
|tt0000014|          7.1|    5023|
|tt0000015|          6.2|     947|
|tt0000016|          5.9|    1325|
|tt0000017|          4.6|     292|
|tt0000018|          5.3|     537|
|tt0000019|          5.3|      28|
|tt0000020|          5.0|     315|
+---------+-------------+--------+
only showing top 20 rows



In [100]:
display(df_ratings)

DataFrame[tconst: string, averageRating: string, numVotes: string]

In [93]:
df_ratings.createOrReplaceTempView('tb_ratings')

In [71]:
spark.sql \
("select  \
*  from  \
tb_ratings a,\
tb_titles b \
 where \
 a.tconst = b.tconst \
 ").show()

+---------+-------------+--------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|   tconst|averageRating|numVotes|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+---------+-------------+--------+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt0000002|          6.0|     233|tt0000002|    short|Le clown et ses c...|Le clown et ses c...|      0|     1892|     \N|             5|     Animation,Short|
|tt0000004|          6.1|     152|tt0000004|    short|         Un bon bock|         Un bon bock|      0|     1892|     \N|            12|     Animation,Short|
|tt0000008|          5.5|    1965|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|   Documentary,Short|
|tt0000010|          6.9|    6530|tt0000010|  

In [111]:
# Join tables         
spark.sql \
("select  \
Count(*)  from  \
tb_ratings a,\
tb_titles b \
 where \
 a.tconst = b.tconst \
 ").show()

+--------+
|count(1)|
+--------+
| 1182639|
+--------+



In [106]:
# Count table title         
spark.sql \
("select  \
Count(*)  from  \
tb_titles  \
 ").show()

+--------+
|count(1)|
+--------+
| 8203690|
+--------+



In [109]:
spark.sql \
("select distinct titleType \
 from  \
tb_titles  \
 ").show()

+------------+
|   titleType|
+------------+
|    tvSeries|
|tvMiniSeries|
|     tvMovie|
|   tvEpisode|
|       movie|
|   tvSpecial|
|       video|
|   videoGame|
|     tvShort|
|       short|
|     tvPilot|
| radioSeries|
|radioEpisode|
+------------+



In [116]:
spark.sql \
("select distinct genres \
 from  \
tb_titles  \
 ").show()

+--------------------+
|              genres|
+--------------------+
|        Comedy,Sport|
|  Action,War,Western|
|Action,Adventure,...|
|Documentary,Drama...|
|Adult,Comedy,Musical|
|  Crime,Horror,Short|
|Animation,Sci-Fi,War|
|Documentary,News,...|
| Adult,Horror,Sci-Fi|
|Fantasy,Horror,Mu...|
|  Action,Adult,Short|
| Music,Musical,Short|
|Animation,Sport,T...|
|Action,Reality-TV...|
| Documentary,Western|
|Adventure,Family,...|
|Comedy,Drama,Western|
|Game-Show,Reality-TV|
|Biography,Reality...|
|Action,Animation,War|
+--------------------+
only showing top 20 rows



In [110]:
# Quantos filmes (incluindo os da televisão) foram lançados no ano de 2015?           
spark.sql \
("select count(*) \
 from  \
tb_titles  \
where startYear = 2015 \
and titleType in ('movie','tvMovie')  \
 ").show()

+--------+
|count(1)|
+--------+
|   19987|
+--------+



In [127]:
# Qual o gênero de títulos mais frequente?          
spark.sql \
("select count(*),genres   \
 from  \
tb_titles  \
group by genres \
order by 1  desc \
LIMIT 1 \
 ").show()

+--------+------+
|count(1)|genres|
+--------+------+
|  880649| Drama|
+--------+------+



In [138]:
# Qual o gênero com a melhor nota média de títulos?      
spark.sql \
("select  \
avg(averageRating),genres from  \
tb_ratings a,\
tb_titles b \
where \
a.tconst = b.tconst \
group by genres \
order by 1  desc \
LIMIT 10 \
").show()

+------------------+--------------------+
|avg(averageRating)|              genres|
+------------------+--------------------+
|               9.8|Comedy,History,We...|
|               9.8|Biography,Comedy,...|
|               9.7|News,Reality-TV,S...|
| 9.409756097560976|Music,Musical,Tal...|
| 9.406666666666666|Adventure,Music,M...|
| 9.290476190476191|Comedy,Game-Show,...|
|              9.25|Biography,Crime,R...|
| 9.209999999999999|Biography,Reality...|
|               9.2|Comedy,Fantasy,Ga...|
|               9.2|Horror,News,Talk-...|
+------------------+--------------------+



In [ ]:
# Qual o gênero com a melhor nota média de títulos?      
spark.sql \
("select  \
avg(averageRating),genres from  \
tb_ratings a,\
tb_titles b \
where \
a.tconst = b.tconst \
group by genres \
order by 1  desc \
LIMIT 10 \
").show()

In [147]:
spark.sql \
("select distinct count(tconst) \
 from  \
tb_titles  \
 ").show()

+-------------+
|count(tconst)|
+-------------+
|      8203690|
+-------------+



In [146]:
print("Distinct Count: " + str(df_titles.distinct().count()))

Distinct Count: 8203690


In [153]:
# Qual a duração média dos filmes com conteúdo adulto? Use uma combinação de filter() e describe().   
spark.sql \
("select  \
avg(runtimeMinutes) from  \
tb_titles b \
where \
b.isAdult = '1' \
LIMIT 10 \
").show()

+-------------------+
|avg(runtimeMinutes)|
+-------------------+
|  92.79938555059914|
+-------------------+



In [154]:
spark.sql \
("select distinct primaryTitle \
 from  \
tb_titles  \
 ").show()

+--------------------+
|        primaryTitle|
+--------------------+
|La gare Saint-Lazare|
|     The X-Ray Fiend|
|La descente de croix|
|   Une rage de dents|
|The Story of the ...|
|    The Prussian Spy|
|        Two Memories|
|      The Wrong Coat|
|The Crime of a Gr...|
|     Dutch Gold Mine|
|In the Sultan's G...|
|The Meeting of th...|
|         Mike's Hero|
|When Two Hearts A...|
|    Master and Pupil|
|          Quo Vadis?|
|   Delayed Proposals|
|The Law and the O...|
|  Youth and Jealousy|
|Dance of the Vampire|
+--------------------+
only showing top 20 rows



In [155]:
spark.sql \
("select distinct originalTitle \
 from  \
tb_titles  \
 ").show()

+--------------------+
|       originalTitle|
+--------------------+
|La gare Saint-Lazare|
|Les quais à Marse...|
|     The X-Ray Fiend|
|La descente de croix|
|   Une rage de dents|
|The Story of the ...|
|    The Prussian Spy|
|        Two Memories|
|      The Wrong Coat|
|     Dutch Gold Mine|
|In the Sultan's G...|
|The Meeting of th...|
|         Mike's Hero|
|When Two Hearts A...|
|    Master and Pupil|
|          Quo Vadis?|
|   Delayed Proposals|
|The Law and the O...|
|  Youth and Jealousy|
|  Elizabeth's Prayer|
+--------------------+
only showing top 20 rows



In [157]:
# Quantos filmes têm o título atual (“primary”) diferente do título original? Use uma combinação de filter, e count(). 
spark.sql \
("select  \
count(1) from  \
tb_titles b \
where \
b.originalTitle <> b.primaryTitle \
").show()

+--------+
|count(1)|
+--------+
|  125056|
+--------+



In [176]:
# Qual o filme que tem o nome mais longo?

#Filter DataFrame by checking the length of a column
from pyspark.sql.functions import col,length,trim

df_titles.orderBy(length(col("primaryTitle")).desc()).show()


+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|    tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|
+----------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+
|tt12985206|    video|Otoko wa chi _ ko...|Otoko wa chi _ ko...|      1|     2020|     \N|            99|               Adult|
|tt13007080|    video|Creampie Voluntee...|Creampie Voluntee...|      1|     2015|     \N|           238|               Adult|
| tt7727908|    video|Yufukuna kurashi ...|Yufukuna kurashi ...|      1|     2013|     \N|           129|         Adult,Drama|
|tt12870546|    video|An Ultra K-Cup Ti...|An Ultra K-Cup Ti...|      1|     2020|     \N|           118|               Adult|
|tt12866892|    video|I'm A Dirty Old M...|I'm A Dirty Old M...|      1|     2016|     \N|            \N|      

In [186]:
# Qual o filme que tem o nome mais longo?
spark.sql \
("select  \
length(primaryTitle),primaryTitle,tconst from  \
tb_titles b \
order by 1 desc \
LIMIT 10 \
").show()

+--------------------+--------------------+----------+
|length(primaryTitle)|        primaryTitle|    tconst|
+--------------------+--------------------+----------+
|                 419|Otoko wa chi _ ko...|tt12985206|
|                 409|Creampie Voluntee...|tt13007080|
|                 408|Yufukuna kurashi ...| tt7727908|
|                 405|An Ultra K-Cup Ti...|tt12870546|
|                 401|I'm A Dirty Old M...|tt12866892|
|                 400|'Oppai dakenara s...| tt7395478|
|                 386|While Horsing Aro...|tt13558536|
|                 383|The Dumb Cunt Is ...|tt13053838|
|                 368|EE.UU. Se replieg...|tt13931914|
|                 366|Kanojo ni furaret...| tt7509356|
+--------------------+--------------------+----------+



In [187]:
df_ratings.describe().show()

+-------+---------+------------------+-----------------+
|summary|   tconst|     averageRating|         numVotes|
+-------+---------+------------------+-----------------+
|  count|  1182639|           1182639|          1182639|
|   mean|     null| 6.917028357766265|973.0778656885153|
| stddev|     null|1.3974964575775946|16275.70904325866|
|    min|tt0000001|               1.0|                5|
|    max|tt9916778|              10.0|          2449517|
+-------+---------+------------------+-----------------+



In [203]:
df_title_ratings.describe().filter("titleType = 'movie'").show()

+-------+---------+---------+------------+-------------+-------+---------+-------+--------------+------+-------------+--------+
|summary|   tconst|titleType|primaryTitle|originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|averageRating|numVotes|
+-------+---------+---------+------------+-------------+-------+---------+-------+--------------+------+-------------+--------+
|    min|tt0000001|    movie|      !Next?|       !Next?|      0|     1874|   1933|             0|Action|          1.0|       5|
+-------+---------+---------+------------+-------------+-------+---------+-------+--------------+------+-------------+--------+



In [208]:
# Gerando estatísticas descritivas da coluna "numVotes"
df_desc = df_title_ratings.describe('numVotes')

# Exibição das estatísticas descritivas
df_desc.show()

+-------+------------------+
|summary|          numVotes|
+-------+------------------+
|  count|           1182639|
|   mean| 973.0778656885153|
| stddev|16275.709043258397|
|    min|                 5|
|    max|           2449517|
+-------+------------------+



In [210]:
# Filtrando o DataFrame original para obter apenas as linhas com 750 votos
df_max_votes = df_title_ratings.filter('numVotes = 2449517') 

# Exibição do resultado
df_max_votes.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+------+-------------+--------+
|tt0111161|    movie|The Shawshank Red...|The Shawshank Red...|      0|     1994|     \N|           142| Drama|          9.3| 2449517|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+------+-------------+--------+



In [211]:
df_max_votes_movies = df_title_ratings.filter("titleType = 'movie'")

# Exibição do resultado
df_max_votes_movies.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|         genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+-------------+--------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|             \N|          4.5|      14|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|          Drama|          3.8|      23|
|tt0000675|    movie|         Don Quijote|         Don Quijote|      0|     1908|     \N|            \N|          Drama|          4.9|      19|
|tt0000739|    movie|El pastorcito de ...|El pastorcito de ...|      0|     1908|     \N|            \N|          Drama|          6.3|  

In [212]:
# Gerando estatísticas descritivas da coluna "numVotes"
df_desc2 = df_max_votes_movies.describe('numVotes')

# Exibição das estatísticas descritivas
df_desc2.show()

+-------+-----------------+
|summary|         numVotes|
+-------+-----------------+
|  count|           267542|
|   mean|3413.902878800338|
| stddev|32827.99730565559|
|    min|                5|
|    max|          2449517|
+-------+-----------------+



In [213]:
# Gerando estatísticas descritivas da coluna "numVotes"
df_desc2 = df_max_votes_movies.describe('averageRating')

# Exibição das estatísticas descritivas
df_desc2.show()

+-------+------------------+
|summary|     averageRating|
+-------+------------------+
|  count|            267542|
|   mean| 6.133833566318403|
| stddev|1.3364291439362925|
|    min|               1.0|
|    max|              10.0|
+-------+------------------+



In [ ]:
# Filtrando o DataFrame original para obter apenas as linhas com 750 votos
df_max_votes = df_title_ratings.filter('numVotes = 2449517')

# Exibição do resultado
df_max_votes.show()

In [202]:
df_title_ratings.filter("titleType = 'movie'").show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|         genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+---------------+-------------+--------+
|tt0000502|    movie|            Bohemios|            Bohemios|      0|     1905|     \N|           100|             \N|          4.5|      14|
|tt0000630|    movie|              Hamlet|              Amleto|      0|     1908|     \N|            \N|          Drama|          3.8|      23|
|tt0000675|    movie|         Don Quijote|         Don Quijote|      0|     1908|     \N|            \N|          Drama|          4.9|      19|
|tt0000739|    movie|El pastorcito de ...|El pastorcito de ...|      0|     1908|     \N|            \N|          Drama|          6.3|  

In [197]:
print("Distinct Count merge: " + str(df_title_ratings.distinct().count()))

Distinct Count merge: 1182639


In [196]:
df_title_ratings.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0000008|    short|Edison Kinetoscop...|Edison Kinetoscop...|      0|     1894|     \N|             1|   Documentary,Short|          5.5|    1965|
|tt0000015|    short| Autour d'une cabine| Autour d'une cabine|      0|     1894|     \N|             2|     Animation,Short|          6.2|     947|
|tt0000019|    short|    The Clown Barber|    The Clown Barber|      0|     1898|     \N|            \N|        Comedy,Short|          5.3|      28|
|tt0000051|    short|The Bohemian Enca...|Campement de bohé...|      0|     1896|     \N|            \N|  